# Airport/Station Video Analytics System
## Real-time Crowd Density & Abandoned Object Detection

**Project Overview:**
- Detect overcrowding in designated areas
- Identify abandoned objects (bags, luggage)
- Generate real-time alerts for security personnel
- Track people density and movement patterns

**Technologies Used:**
- YOLOv8 for object detection
- OpenCV for video processing
- Background subtraction for abandoned object detection
- Real-time alert system

## 1. Setup and Installation

In [1]:
# Install required packages
!pip install ultralytics opencv-python-headless numpy matplotlib pillow
!pip install supervision scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.4/217.4 kB 5.7 MB/s eta 0:00:00


In [2]:
# Import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from collections import defaultdict, deque
from datetime import datetime
import time
from IPython.display import clear_output, Image, display
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ All libraries imported successfully!


## 2. Configuration and Parameters

In [3]:
# Configuration Parameters
class Config:
    # Model settings
    MODEL_NAME = 'yolov8n.pt'
    CONFIDENCE_THRESHOLD = 0.5
    IOU_THRESHOLD = 0.45

    # Crowd detection settings
    OVERCROWDING_THRESHOLD = 15
    HIGH_DENSITY_THRESHOLD = 10

    # Abandoned object detection settings
    ABANDONMENT_TIME = 30
    PROXIMITY_THRESHOLD = 100
    STATIC_OBJECT_TIME = 10

    # Background subtraction
    BG_HISTORY = 500
    BG_THRESHOLD = 16
    BG_LEARNING_RATE = 0.01

    # Visualization
    FONT = cv2.FONT_HERSHEY_SIMPLEX
    FONT_SCALE = 0.6
    THICKNESS = 2

    # Alert colors (BGR format)
    COLOR_SAFE = (0, 255, 0)
    COLOR_WARNING = (0, 165, 255)
    COLOR_DANGER = (0, 0, 255)
    COLOR_ABANDONED = (255, 0, 255)

config = Config()
print("✅ Configuration loaded!")

✅ Configuration loaded!


## 3. Load YOLO Model

In [4]:
# Load YOLOv8 model
print("Loading YOLOv8 model...")
model = YOLO(config.MODEL_NAME)
print(f"✅ Model loaded: {config.MODEL_NAME}")

# Get class names
class_names = model.names
print(f"\nDetectable classes: {len(class_names)}")
print(f"Person class ID: {list(class_names.values()).index('person')}")

Loading YOLOv8 model...
✅ Model loaded: yolov8n.pt

Detectable classes: 80
Person class ID: 0


## 4. Define Region of Interest (ROI)

In [5]:
class ROI:
    def __init__(self, name, polygon_points):
        self.name = name
        self.polygon = np.array(polygon_points, np.int32)
        self.person_count = 0
        self.status = "SAFE"
        self.color = config.COLOR_SAFE

    def is_point_inside(self, point):
        return cv2.pointPolygonTest(self.polygon, point, False) >= 0

    def update_status(self):
        if self.person_count >= config.OVERCROWDING_THRESHOLD:
            self.status = "OVERCROWDED"
            self.color = config.COLOR_DANGER
        elif self.person_count >= config.HIGH_DENSITY_THRESHOLD:
            self.status = "HIGH DENSITY"
            self.color = config.COLOR_WARNING
        else:
            self.status = "SAFE"
            self.color = config.COLOR_SAFE
        return self.status

def create_default_rois(frame_width, frame_height):
    rois = [
        ROI("Checkpoint Area", [
            [int(frame_width * 0.1), int(frame_height * 0.3)],
            [int(frame_width * 0.5), int(frame_height * 0.3)],
            [int(frame_width * 0.5), int(frame_height * 0.7)],
            [int(frame_width * 0.1), int(frame_height * 0.7)]
        ]),
        ROI("Waiting Area", [
            [int(frame_width * 0.5), int(frame_height * 0.3)],
            [int(frame_width * 0.9), int(frame_height * 0.3)],
            [int(frame_width * 0.9), int(frame_height * 0.7)],
            [int(frame_width * 0.5), int(frame_height * 0.7)]
        ])
    ]
    return rois

print("✅ ROI class defined!")

✅ ROI class defined!


## 5. Abandoned Object Detection System

In [6]:
class AbandonedObjectDetector:
    def __init__(self):
        self.tracked_objects = {}
        self.abandoned_objects = {}
        self.next_id = 0
        self.trackable_classes = ['backpack', 'handbag', 'suitcase', 'umbrella', 'bottle']

    def calculate_distance(self, point1, point2):
        return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

    def is_owner_nearby(self, obj_position, person_positions):
        for person_pos in person_positions:
            if self.calculate_distance(obj_position, person_pos) < config.PROXIMITY_THRESHOLD:
                return True
        return False

    def update(self, detections, person_positions, current_time):
        current_objects = []

        for bbox, class_name in detections:
            if class_name in self.trackable_classes:
                x1, y1, x2, y2 = bbox
                center = ((x1 + x2) // 2, (y1 + y2) // 2)
                current_objects.append((center, class_name, bbox))

        matched_ids = set()
        for center, class_name, bbox in current_objects:
            best_match = None
            best_distance = float('inf')

            for obj_id, obj_data in self.tracked_objects.items():
                if obj_id in matched_ids:
                    continue
                distance = self.calculate_distance(center, obj_data['position'])
                if distance < 50 and distance < best_distance:
                    best_distance = distance
                    best_match = obj_id

            if best_match is not None:
                matched_ids.add(best_match)
                obj_data = self.tracked_objects[best_match]

                if self.calculate_distance(center, obj_data['position']) < 10:
                    obj_data['static_time'] = current_time - obj_data['first_seen']
                else:
                    obj_data['first_seen'] = current_time
                    obj_data['static_time'] = 0

                obj_data['position'] = center
                obj_data['bbox'] = bbox
                obj_data['last_seen'] = current_time
                obj_data['owner_nearby'] = self.is_owner_nearby(center, person_positions)

                if (obj_data['static_time'] >= config.STATIC_OBJECT_TIME and
                    not obj_data['owner_nearby'] and
                    (current_time - obj_data['last_owner_time']) >= config.ABANDONMENT_TIME):
                    if best_match not in self.abandoned_objects:
                        self.abandoned_objects[best_match] = {
                            'position': center,
                            'bbox': bbox,
                            'class': class_name,
                            'abandoned_time': current_time
                        }
            else:
                owner_nearby = self.is_owner_nearby(center, person_positions)
                self.tracked_objects[self.next_id] = {
                    'position': center,
                    'bbox': bbox,
                    'class': class_name,
                    'first_seen': current_time,
                    'last_seen': current_time,
                    'static_time': 0,
                    'owner_nearby': owner_nearby,
                    'last_owner_time': current_time
                }
                self.next_id += 1

        for obj_id, obj_data in self.tracked_objects.items():
            if obj_data['owner_nearby']:
                obj_data['last_owner_time'] = current_time

        to_remove = []
        for obj_id, obj_data in self.tracked_objects.items():
            if current_time - obj_data['last_seen'] > 5:
                to_remove.append(obj_id)

        for obj_id in to_remove:
            del self.tracked_objects[obj_id]
            if obj_id in self.abandoned_objects:
                del self.abandoned_objects[obj_id]

        return self.abandoned_objects

print("✅ Abandoned Object Detector class defined!")

✅ Abandoned Object Detector class defined!


## 6. Alert System

In [7]:
class AlertSystem:
    def __init__(self):
        self.alerts = []
        self.alert_cooldown = {}

    def add_alert(self, alert_type, message, severity="INFO"):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        alert_key = f"{alert_type}_{message}"
        current_time = time.time()

        if alert_key in self.alert_cooldown:
            if current_time - self.alert_cooldown[alert_key] < 10:
                return

        self.alert_cooldown[alert_key] = current_time

        alert = {
            'timestamp': timestamp,
            'type': alert_type,
            'message': message,
            'severity': severity
        }
        self.alerts.append(alert)

        severity_symbol = "⚠️" if severity == "WARNING" else "🚨" if severity == "CRITICAL" else "ℹ️"
        print(f"{severity_symbol} [{timestamp}] {alert_type}: {message}")

    def get_recent_alerts(self, n=5):
        return self.alerts[-n:] if len(self.alerts) >= n else self.alerts

alert_system = AlertSystem()
print("✅ Alert System initialized!")

✅ Alert System initialized!


## 7. Main Video Analytics Pipeline

In [8]:
def draw_dashboard(frame, rois, total_people, abandoned_count, fps, alerts):
    h, w = frame.shape[:2]

    overlay = frame.copy()
    cv2.rectangle(overlay, (10, 10), (400, 200), (0, 0, 0), -1)
    frame = cv2.addWeighted(overlay, 0.7, frame, 0.3, 0)

    cv2.putText(frame, "Airport Security Analytics", (20, 35),
                config.FONT, 0.7, (255, 255, 255), 2)

    cv2.putText(frame, f"Total People: {total_people}", (20, 65),
                config.FONT, 0.5, (255, 255, 255), 1)
    cv2.putText(frame, f"Abandoned Objects: {abandoned_count}", (20, 90),
                config.FONT, 0.5, (255, 100, 100) if abandoned_count > 0 else (255, 255, 255), 1)
    cv2.putText(frame, f"FPS: {fps:.1f}", (20, 115),
                config.FONT, 0.5, (255, 255, 255), 1)

    y_offset = 140
    for roi in rois:
        status_text = f"{roi.name}: {roi.person_count} ({roi.status})"
        cv2.putText(frame, status_text, (20, y_offset),
                    config.FONT, 0.4, roi.color, 1)
        y_offset += 20

    if alerts:
        alert_overlay = frame.copy()
        cv2.rectangle(alert_overlay, (w - 410, h - 110), (w - 10, h - 10), (0, 0, 0), -1)
        frame = cv2.addWeighted(alert_overlay, 0.7, frame, 0.3, 0)

        cv2.putText(frame, "Recent Alerts:", (w - 400, h - 90),
                    config.FONT, 0.5, (255, 255, 255), 1)

        for idx, alert in enumerate(alerts[-3:]):
            alert_text = f"{alert['type'][:20]}..."
            color = (0, 255, 255) if alert['severity'] == "WARNING" else (0, 0, 255)
            cv2.putText(frame, alert_text, (w - 400, h - 65 + idx * 20),
                        config.FONT, 0.4, color, 1)

    return frame

In [9]:
def process_video(video_path, output_path=None, max_frames=None):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"❌ Error: Cannot open video file {video_path}")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps_video = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"\n📹 Video Info:")
    print(f"   Resolution: {frame_width}x{frame_height}")
    print(f"   FPS: {fps_video}")
    print(f"   Total Frames: {total_frames}")

    rois = create_default_rois(frame_width, frame_height)
    abandoned_detector = AbandonedObjectDetector()

    writer = None
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_path, fourcc, fps_video, (frame_width, frame_height))

    frame_count = 0
    start_time = time.time()
    fps_deque = deque(maxlen=30)

    print("\n🚀 Starting video processing...\n")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            if max_frames and frame_count > max_frames:
                break

            frame_start = time.time()
            current_time = time.time() - start_time

            results = model(frame, conf=config.CONFIDENCE_THRESHOLD,
                          iou=config.IOU_THRESHOLD, verbose=False)[0]

            person_positions = []
            object_detections = []

            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                class_name = class_names[cls]

                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2

                if class_name == 'person':
                    person_positions.append((center_x, center_y))
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"Person {conf:.2f}", (x1, y1 - 10),
                               config.FONT, 0.5, (0, 255, 0), 1)
                else:
                    object_detections.append(((x1, y1, x2, y2), class_name))
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 0), 2)
                    cv2.putText(frame, f"{class_name} {conf:.2f}", (x1, y1 - 10),
                               config.FONT, 0.4, (255, 255, 0), 1)

            abandoned_objects = abandoned_detector.update(
                object_detections, person_positions, current_time
            )

            for obj_id, obj_data in abandoned_objects.items():
                x1, y1, x2, y2 = obj_data['bbox']
                cv2.rectangle(frame, (x1, y1), (x2, y2), config.COLOR_ABANDONED, 3)
                cv2.putText(frame, "ABANDONED!", (x1, y1 - 25),
                           config.FONT, 0.6, config.COLOR_ABANDONED, 2)

                time_abandoned = int(current_time - obj_data['abandoned_time'])
                cv2.putText(frame, f"{time_abandoned}s", (x1, y1 - 5),
                           config.FONT, 0.5, config.COLOR_ABANDONED, 1)

                alert_system.add_alert(
                    "ABANDONED OBJECT",
                    f"{obj_data['class']} at ({obj_data['position'][0]}, {obj_data['position'][1]})",
                    "CRITICAL"
                )

            for roi in rois:
                roi.person_count = 0
                for px, py in person_positions:
                    if roi.is_point_inside((px, py)):
                        roi.person_count += 1
                        cv2.circle(frame, (px, py), 5, roi.color, -1)

                status = roi.update_status()
                cv2.polylines(frame, [roi.polygon], True, roi.color, 2)

                label_pos = tuple(roi.polygon[0])
                cv2.putText(frame, f"{roi.name}: {roi.person_count}", label_pos,
                           config.FONT, 0.6, roi.color, 2)

                if status == "OVERCROWDED":
                    alert_system.add_alert(
                        "OVERCROWDING",
                        f"{roi.name} has {roi.person_count} people (threshold: {config.OVERCROWDING_THRESHOLD})",
                        "CRITICAL"
                    )
                elif status == "HIGH DENSITY":
                    alert_system.add_alert(
                        "HIGH DENSITY",
                        f"{roi.name} has {roi.person_count} people",
                        "WARNING"
                    )

            frame_time = time.time() - frame_start
            fps_deque.append(1.0 / frame_time if frame_time > 0 else 0)
            avg_fps = sum(fps_deque) / len(fps_deque)

            frame = draw_dashboard(
                frame, rois, len(person_positions),
                len(abandoned_objects), avg_fps,
                alert_system.get_recent_alerts()
            )

            if writer:
                writer.write(frame)

            if frame_count % 30 == 0:
                print(f"\rProcessed: {frame_count}/{total_frames if not max_frames else max_frames} frames | "
                      f"FPS: {avg_fps:.1f} | People: {len(person_positions)} | "
                      f"Abandoned: {len(abandoned_objects)}", end="")

    except KeyboardInterrupt:
        print("\n\n⚠️ Processing interrupted by user")

    finally:
        cap.release()
        if writer:
            writer.release()

        print(f"\n\n✅ Processing complete!")
        print(f"   Processed {frame_count} frames")
        print(f"   Total alerts generated: {len(alert_system.alerts)}")
        if output_path:
            print(f"   Output saved to: {output_path}")

print("✅ Video processing pipeline defined!")

✅ Video processing pipeline defined!


## 8. Upload Test Video

In [10]:
from google.colab import files
import os

print("Upload your video file:")
uploaded = files.upload()

video_path = list(uploaded.keys())[0]
print(f"✅ Video uploaded: {video_path}")

Upload your video file:


Saving 13606911-uhd_3840_2160_24fps.mp4 to 13606911-uhd_3840_2160_24fps.mp4
✅ Video uploaded: 13606911-uhd_3840_2160_24fps.mp4


## 9. Run Video Analytics

In [11]:
output_video_path = "output_analyzed.mp4"

process_video(
    video_path=video_path,
    output_path=output_video_path,
    max_frames=None
)


📹 Video Info:
   Resolution: 2560x1440
   FPS: 23
   Total Frames: 349

🚀 Starting video processing...

Processed: 330/349 frames | FPS: 9.1 | People: 8 | Abandoned: 0

✅ Processing complete!
   Processed 349 frames
   Total alerts generated: 0
   Output saved to: output_analyzed.mp4


## 10. Download Results

In [12]:
if os.path.exists(output_video_path):
    files.download(output_video_path)
    print(f"✅ Download started: {output_video_path}")
else:
    print("❌ Output video not found")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download started: output_analyzed.mp4


## 11. Generate Alert Report

In [14]:
import pandas as pd

if alert_system.alerts:
    df_alerts = pd.DataFrame(alert_system.alerts)

    print("\n📊 ALERT SUMMARY")
    print("=" * 70)
    print(f"Total Alerts: {len(df_alerts)}")
    print(f"\nAlerts by Type:")
    print(df_alerts['type'].value_counts())
    print(f"\nAlerts by Severity:")
    print(df_alerts['severity'].value_counts())

    print("\n\n📋 RECENT ALERTS (Last 10)")
    print("=" * 70)
    print(df_alerts[['timestamp', 'type', 'message', 'severity']].tail(10).to_string(index=False))

    report_path = "alert_report.csv"
    df_alerts.to_csv(report_path, index=False)
    print(f"\n✅ Alert report saved to: {report_path}")

    files.download(report_path)
else:
    print("No alerts generated during processing")

No alerts generated during processing


## 12. Visualize Statistics

In [15]:
if alert_system.alerts:
    df = pd.DataFrame(alert_system.alerts)

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    df['type'].value_counts().plot(kind='bar', ax=axes[0], color='steelblue')
    axes[0].set_title('Alert Types Distribution', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Alert Type')
    axes[0].set_ylabel('Count')
    axes[0].tick_params(axis='x', rotation=45)

    severity_colors = {'INFO': 'green', 'WARNING': 'orange', 'CRITICAL': 'red'}
    severity_counts = df['severity'].value_counts()
    colors = [severity_colors.get(sev, 'gray') for sev in severity_counts.index]
    severity_counts.plot(kind='pie', ax=axes[1], autopct='%1.1f%%', colors=colors)
    axes[1].set_title('Alert Severity Distribution', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('')

    plt.tight_layout()
    plt.savefig('alert_statistics.png', dpi=150, bbox_inches='tight')
    plt.show()

    print("\n✅ Visualization saved as alert_statistics.png")
    files.download('alert_statistics.png')

## 13. Project Summary

In [16]:
summary_text = f"Airport/Station Video Analytics System\n"
summary_text += f"=====================================\n\n"
summary_text += f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
summary_text += f"Model: YOLOv8n (Nano)\n"
summary_text += f"Framework: Ultralytics YOLO + OpenCV\n\n"
summary_text += f"CAPABILITIES:\n"
summary_text += f"- Real-time people detection and counting\n"
summary_text += f"- Crowd density monitoring with zone-based analysis\n"
summary_text += f"- Overcrowding detection with configurable thresholds\n"
summary_text += f"- Abandoned object detection (bags, luggage, etc.)\n"
summary_text += f"- Automated alert system with severity levels\n"
summary_text += f"- Multi-zone Region of Interest (ROI) monitoring\n\n"
summary_text += f"Total Alerts Generated: {len(alert_system.alerts)}\n"

print(summary_text)

with open('project_summary.txt', 'w') as f:
    f.write(summary_text)

print("\n✅ Project summary saved!")
files.download('project_summary.txt')

Airport/Station Video Analytics System

Date: 2026-02-16 12:07:11
Model: YOLOv8n (Nano)
Framework: Ultralytics YOLO + OpenCV

CAPABILITIES:
- Real-time people detection and counting
- Crowd density monitoring with zone-based analysis
- Overcrowding detection with configurable thresholds
- Abandoned object detection (bags, luggage, etc.)
- Automated alert system with severity levels
- Multi-zone Region of Interest (ROI) monitoring

Total Alerts Generated: 0


✅ Project summary saved!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 🎓 Learning Resources & Next Steps

### Key Concepts Covered:
1. **Object Detection**: YOLOv8 for real-time detection
2. **Object Tracking**: Proximity-based tracking
3. **Spatial Analysis**: ROI-based zone monitoring
4. **Temporal Analysis**: Time-based abandonment detection
5. **Alert Systems**: Multi-level severity alerts

### Future Enhancements:
- Add DeepSORT for better object tracking
- Implement trajectory analysis
- Add behavior recognition (fighting, running)
- Integrate with databases (MongoDB, PostgreSQL)
- Build REST API with Flask/FastAPI
- Create web dashboard with real-time updates
- Add email/SMS notifications
- Implement multi-camera fusion

### Deployment Options:
- **Edge**: NVIDIA Jetson Nano/Xavier
- **Cloud**: AWS/GCP/Azure with Docker
- **Hybrid**: Edge detection + Cloud analytics